In [7]:
# setup library imports
import io, time, json
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pickle
from collections import Counter
import html
from datetime import datetime

In [79]:
#helper function to locate the location of interest
def find_helper(root, tag, content, ty='movie'):
    hs=root.find_all(tag)
    details=None
    for h in hs:
        if h.get_text()==content:
            if ty != 'person':
                details = h.find_next()
            else:
                 details = h.find_next().find_next()
            break
            
    return details
    pass

In [80]:
#helper function to parse the table
def table_helper(root, titles, tag, content):
    table_details = find_helper(root, tag, content)
    
    #get the table list
    table_list=list()
    
    if table_details==None:
        return table_list
    else:
        table_details=table_details.table
        if table_details==None:
            return table_list
        
        
    for row in table_details.find_all("tr"):
        i=0
        table_dic=dict()
        for column in row.find_all("td"):
            if titles[i]!='Date':
                if titles[i]=='Change':
                    if len(table_list)==0:
                        table_dic['Change']=0.0
                    else:
                        try:
                            table_dic['Change']=float(column.get_text().replace(u'\xa0', u' ').strip('%'))/100
                        except:
                            table_dic['Change']=float(0.0)
                else:
                    if column.get_text().replace(u'\xa0', u' ')=='-':
                        table_dic[titles[i]]='-'
                    else:
                        try:
                            table_dic[titles[i]]=int(column.get_text().replace(u'\xa0', u' ').replace('$','').replace(',',''))
                        except:
                            table_dic[titles[i]]=html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
                            pass
            else:
                table_dic[titles[i]]=datetime.strptime(html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r',''), '%Y/%m/%d')
            i+=1
                
        if len(table_dic)>0:
            table_list.append(table_dic)
                
    return table_list

In [81]:
#helper function to parse the center table
def center_table_helper(root, titles, tag, content, ty='movie'):
    table_details = find_helper(root, tag, content, ty)
        
    #get the center table list
    table_list=list()
    
    if table_details==None:
        return table_list
    else:
        table_details=table_details.table
        if table_details==None:
            return table_list
        
    for row in table_details.find_all("tr"):
        i=0
        isend=False
        table_dic=dict()
        if len(row.find_all("td"))==len(titles):
            for column in row.find_all("td"):
                if titles[i]=='Chart Date':
                    try:
                        table_dic[titles[i]]=datetime.strptime(html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','').strip(), '%b %d, %Y')
                    except:
                        print(column.get_text())
                        isend=True
                        break
                elif titles[i]=='Release Date' or titles[i]=='Report Date':
                    try:
                        table_dic[titles[i]]=datetime.strptime(html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r',''), '%m/%d/%Y')
                    except:
                        print(column.get_text())
                        isend=True
                        break
                elif titles[i]=='Domestic Share':
                    try:
                        table_dic[titles[i]]=float(html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('%',''))/100
                    except:
                        table_dic[titles[i]]=0.0
                elif (not titles[i].startswith('Record'))  and (not titles[i].startswith('Role')) and (not titles[i].startswith('Title')) and (not titles[i].startswith('Report')) and (not titles[i].startswith('Chart')) and (not titles[i].startswith('Territory')) and (not titles[i].startswith('Release')) and titles[i]!='':
                    if column.get_text().replace(u'\xa0', u' ')=='-':
                        table_dic[titles[i]]=-1
                    else:
                        try:
                            table_dic[titles[i]]=int(column.get_text().replace(u'\xa0', u' ').replace('$','').replace(',',''))
                        except:
                            table_dic[titles[i]]=html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
                            pass
                else:
                    if titles[i]!='':
                        table_dic[titles[i]]=html.unescape(column.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
                i+=1
        if isend:
            break
        if len(table_dic)>0:
            table_list.append(table_dic)
    
    return table_list

In [82]:
#helper function to parse the table in cast tab
def cast_table_helper(root, tag, content):
    details=find_helper(root, tag, content)
    host_url="https://www.the-numbers.com"
    
    is_First=True
    table_list=list()
    
    if details==None:
        return table_list
    
    for a in details.find_all('a'):
        table_dic=dict()
        table_dic['name']=html.unescape(a.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','').strip()
        table_dic['url']=host_url+a.get('href')
        if content=='Production and Technical Credits':
            if is_First:
                table_dic['role']=html.unescape(a.find_next().find_next().find_next().get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','').strip()
                is_First=False
            else:
                table_dic['role']=html.unescape(a.find_next().find_next().get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','').strip()
        table_list.append(table_dic)
    
    return table_list

In [83]:
#helper function
#given the html content, return the information as a dictionary
def parse_page(html_page, title):
    root=BeautifulSoup(html_page.decode('utf-8'), "html.parser")
    lis=dict()
    if title=="summary":
        #get the table tag of the movie details
        details = find_helper(root, 'h2', 'Movie Details')
        
        if details==None:
            return lis
        
        #get the information of the movie
        for item in details.find_all("tr"):
            first_child = item.contents[0]
            second_child = first_child.find_next_sibling()
            c_name = html.unescape(first_child.b.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
            if c_name.endswith('Budget:'):
                try:
                    lis['Budget'] = int(second_child.get_text().replace(u'\xa0', u' ').replace('\n','').replace('\r','').replace('$','').replace(',',''))
                except:
                    lis['Budget'] = html.unescape(second_child.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
                    pass
                
            elif c_name.startswith("Domestic"):
                text=second_child.prettify().split('<br/>')
                sub_dic=dict()
                for cate in text:
                    cate_temp = cate.split(')')[0].split('(')
                    date_data=html.unescape(cate_temp[0]).replace('<td>','').replace('\n','').replace('\r','').replace('by','').strip()
                    if len(date_data.split(','))>1:
                        date_type=date_data.split(',')[0][-2:]
                        try:
                            sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data, '%B %d'+date_type+', %Y')
                        except:
                            sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data, '%B, %Y')
                    else:
                        if len(date_data.split())==1:
                            sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data, '%Y')
                        else:
                            sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data.split()[1], '%Y')
                    
                lis['Domestic Releases'] = sub_dic
                
            elif c_name.startswith('International'):
                text = second_child.get_text().replace(u'\xa0', u' ').split('\n')
                sub_dic=dict()
                for cate in text:
                    cate_temp = cate.replace(')','').split('(')
                    if len(cate_temp)>2:
                        temp_key = cate_temp[2].strip()
                        if temp_key in sub_dic.keys():
                            temp_dic=sub_dic[temp_key]
                        else:
                            temp_dic=dict()
                        
                        date_data=html.unescape(cate_temp[0]).replace('<td>','').replace(u'\xa0', u' ').replace('\n','').replace('\r','').replace('Week of','').strip()
                        if len(date_data.split(','))>1:
                            date_type=date_data.split(',')[0][-2:]
                            try:
                                sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data, '%B %d'+date_type+', %Y')
                            except:
                                sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data, '%B, %Y')
                        else:
                            sub_dic[cate_temp[1].strip()] = datetime.strptime(date_data, '%Y')
                
                lis['International Releases']=sub_dic
            
            elif c_name.startswith('MPAA'):
                lis['MPAA Rating']=html.unescape(second_child.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
            
            elif c_name.startswith('Comparisons'):
                pass
            
            elif c_name.startswith('Keywords'):
                lis['Keywords']=second_child.get_text().replace(u'\xa0', u' ').split(',')
            
            elif c_name.startswith('Creative'):
                lis['Creative Type']=html.unescape(second_child.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
                
            elif c_name.startswith('Production Countries') or c_name.startswith('Production Companies'):
                lis[c_name.replace(':','')]=html.unescape(second_child.get_text()).replace(u'\xa0', u' ').split(',')
            
            elif c_name.startswith('Production Method'):
                lis['Production Method']=second_child.get_text().replace(u'\xa0', u' ').split('/')
                 
            else:
                lis[c_name.replace(':','')]=html.unescape(second_child.get_text()).replace(u'\xa0', u' ').replace('\n','').replace('\r','')
            pass
        

        #The column name
        title_name=['Record', 'Rank', 'Amount', 'Chart Date', 'Days In Release']
        
        #get the rank list
        lis['ranking on other records']=center_table_helper(root, title_name, 'h3', 'Ranking on other Records and Milestones')
                
    
    elif title=="box_office":
        #The column name
        title_name=['Record', 'Rank', 'Revenue']
        #get the demostic table information
        lis['demostic']=center_table_helper(root, title_name, 'h2', 'Domestic Cumulative Box Office Records')
        
        #get the weekend box office performance
        weekend_title=['Date', 'Rank', 'Gross', 'Change', 'Theaters', 'Per Theater', 'Total Gross', 'Week']
        lis['weekend']=table_helper(root, weekend_title, 'h2', 'Weekend Box Office Performance')
        
        #get the daliy box office performance
        daliy_title=['Date', 'Rank', 'Gross', 'Change', 'Theaters', 'Per Theater', 'Total Gross', 'Day']
        lis['daliy']=table_helper(root, weekend_title, 'h2', 'Daily Box Office Performance')
        
        #get the weekly box office performanc
        weekly_title=['Date', 'Rank', 'Gross', 'Change', 'Theaters', 'Per Theater', 'Total Gross', 'Week']
        
        lis['weekly']=table_helper(root, weekend_title, 'h2', 'Weekly Box Office Performance')
        pass
    
    
    elif title=="international":
        title_name=['Territory', 'Release Date', 'Opening Weekend', 'Opening Weekend Theaters', 'Maximum Theaters', 'Theatrical Engagements', 'Total Box Office', 'Report Date']
        #get the box office summary
        lis['Box Office Summary']= center_table_helper(root, title_name, 'h2','Box Office Summary Per Territory')
        
        title_name=['Record', 'Rank', 'Revenue']
        #get the international box office
        lis['International Box Office']=center_table_helper(root, title_name, 'h2', 'International Cumulative Box Office Records')
        
        title_name=['Record', 'Rank', 'Revenue']
        lis['Worldwide Box Office']=center_table_helper(root, title_name, 'h2', 'Worldwide Cumulative Box Office Records')
        pass
    
    
    elif title=="news":
        host_url="https://www.the-numbers.com"
        url_list=list()
        for item in root.find_all('a'):
            if item.get_text().replace(u'\xa0', u' ')=='More...':
                url_list.append(host_url+item.get('href'))
                
        text_list=list()
        for url in url_list:
            url_response=requests.get(url)
            url_html=url_response.content
            url_root=BeautifulSoup(url_html.decode('utf-8'), "html.parser")
            for div in url_root.find_all('div'):
                if div.get('id')=='news':
                    news_content=html.unescape(div.get_text()).replace('\n',' ').replace('\r', '').replace(u'\xa0', u' ')
                    text_list.append(news_content)
                    break

        lis['news']=text_list
        pass
    
    elif title=="cast":
        lis['leading_members']=cast_table_helper(root, 'h1', 'Lead Ensemble Members')
        lis['supporting_cast']=cast_table_helper(root, 'h1', 'Supporting Cast')
        lis['cameos']=cast_table_helper(root, 'h1', 'Cameos')
        lis['uncategorized']=cast_table_helper(root, 'h1', 'Uncategorized')
        lis['production']=cast_table_helper(root, 'h1', 'Production and Technical Credits')
        pass
            
    return lis

In [84]:
#helper function
#given the url of the movie, get all the information of the movie
#https://www.the-numbers.com/movie/Black-Panther#tab=summary
def get_info(url):
    summary_url=url+"#tab=summary"
    international_url=url+"#tab=international"
    box_office_url=url+"#tab=box-office"
    news_url=url+"#tab=news"
    cast_url=url+"#tab=cast-and-crew"
    
    summary_response = requests.get(summary_url)
    international_response = requests.get(international_url)
    box_office_response = requests.get(box_office_url)
    news_response = requests.get(news_url)
    cast_response = requests.get(cast_url)
    summary_dic, international_dic, box_office_dic, news_dic, cast_dic = dict(), dict(), dict(), dict(), dict()
    
    if summary_response.status_code != 404:
        summary_dic = parse_page(summary_response.content, "summary")
        #pprint(summary_dic)
    
    if international_response.status_code != 404:
        international_dic = parse_page(international_response.content, "international")
        #pprint(international_dic)
    
    if box_office_response.status_code != 404:
        box_office_dic = parse_page(box_office_response.content, "box_office")
        #pprint(box_office_dic)
        
    if news_response.status_code !=404:
        news_dic=parse_page(news_response.content, "news")
        ##print(len(news_dic['news']))
        #pprint(news_dic)
        
    if cast_response.status_code!=404:
        cast_dic=parse_page(cast_response.content, "cast")
        #pprint(cast_dic)
        
    total_dic=dict()
    total_dic['summary']=summary_dic
    total_dic['international']=international_dic
    total_dic['box_office']=box_office_dic
    total_dic['news']=news_dic
    total_dic['cast']=cast_dic
    
    return total_dic

In [15]:
#example to use the get_info funciton
url="https://www.the-numbers.com/movie/Black-Panther"
final_dic=get_info(url)
pprint(final_dic)

{'box_office': {'daliy': [{'Change': 0.0,
                           'Date': datetime.datetime(2018, 2, 16, 0, 0),
                           'Gross': 75941146,
                           'Per Theater': 18891,
                           'Rank': 1,
                           'Theaters': 4020,
                           'Total Gross': 75941146,
                           'Week': 1},
                          {'Change': -0.13,
                           'Date': datetime.datetime(2018, 2, 17, 0, 0),
                           'Gross': 65995366,
                           'Per Theater': 16417,
                           'Rank': 1,
                           'Theaters': 4020,
                           'Total Gross': 141936512,
                           'Week': 2},
                          {'Change': -0.09,
                           'Date': datetime.datetime(2018, 2, 18, 0, 0),
                           'Gross': 60067439,
                           'Per Theater': 14942,
                 

                             'Rank': 8,
                             'Theaters': 1930,
                             'Total Gross': 681374736,
                             'Week': 10}],
                'weekly': [{'Change': 0.0,
                            'Date': datetime.datetime(2018, 2, 16, 0, 0),
                            'Gross': 291954422,
                            'Per Theater': 72625,
                            'Rank': 1,
                            'Theaters': 4020,
                            'Total Gross': 291954422,
                            'Week': 1},
                           {'Change': -0.51,
                            'Date': datetime.datetime(2018, 2, 23, 0, 0),
                            'Gross': 143445615,
                            'Per Theater': 35683,
                            'Rank': 1,
                            'Theaters': 4020,
                            'Total Gross': 435400037,
                            'Week': 2},
                         

                                                           'International Box '
                                                           'Office for Based '
                                                           'on Comic/Graphic '
                                                           'Novel Movies',
                                                 'Revenue': 642073119},
                                                {'Rank': 14,
                                                 'Record': 'All Time '
                                                           'International Box '
                                                           'Office for Live '
                                                           'Action Movies',
                                                 'Revenue': 642073119},
                                                {'Rank': 5,
                                                 'Record': 'All Time '
                                              

                   'market. It bombed too hard in theaters for word-of-mouth '
                   'to matter there.   Unsane was right behind with $3.76 '
                   'million in 2,023 theaters. It too missed the Mendoza Line '
                   '($2,000/theater), so it will also disappear from theaters '
                   'shortly. That’s too bad, as it earned the best reviews, '
                   'even though it only managed a B minus from CinemaScore. '
                   "That said, a B minus isn't that bad for a horror movie.    "
                   'Moving on to the sophomore class, we find Love, Simon in '
                   'seventh place with $7.61 million over the weekend for a '
                   'two-week total of $23.50 million. It only fell 35%, so it '
                   'should stick around in theaters long enough to hit $40 '
                   'million, which is more than enough to be a hit for a film '
                   'that cost $10 million to make. On 

                   'hand.     Red Sparrow opened in second place with $6.03 '
                   'million, but its reviews are mixed and it only managed a B '
                   'from CinemaScore, so it likely won’t have great legs. We '
                   'predicted $18 million, but $16 million is more likely at '
                   'this point.   Death Wish was even weaker with an opening '
                   'day of $4.28 million. Its reviews are even worse, but it '
                   'did earn a B plus from CinemaScore. Look for an opening '
                   'weekend of $12 million, which is inline with our '
                   'adjustedexpectations.     - Friday Box Office Chart    - '
                   'Red Sparrow Comparisons  - Death Wish Comparisons   '
                   'C.S.Strowbridge ',
                   '  Weekend Predictions: New Releases Battle for Season '
                   'Place as Black Panther Completes Threepeat March 1st, '
                   '2018        

                   'Written By: Zachary Warren Smith, Jason Baumgardner, Galen '
                   'Gilbert, Timothy Ratajczak, Jason Baumgardner, Zachary '
                   'Warren Smith Starring: Caitlin Leahy, Taylor James '
                   'Production Budget: Unknown - Estimated at $10 million to '
                   '$20 million  Box Office Potential: $15 million    A Bible '
                   'Epic from Pure Flix Entertainment. Bible Epics used to be '
                   'a reliable box office draw, but that was a long, long time '
                   'ago. Furthermore, the biggest Pure Flix hit outside the '
                   'God’s Not Dead franchise earned less than $15 million. '
                   'This film only has a 50/50 chance to beat that mark. On '
                   'the positive side, it likely didn’t cost a lot to make.    '
                   'Weekend of February 23rd, 2018       The final weekend of '
                   'the month is a post-holiday weekend

In [85]:
#function to get the dictionary of {name:url}
#The result is saved in name_url.pkl, no need to run it
#https://www.the-numbers.com/movie/budgets/all/5501
#The structure of the dictionary is as following:
#{name of movie: {'url': string,
#                  'Release Date': datetime,
#                  'Production Budget': int,
#                  'Domestic Gross': int, 
#                   'Worldwide Gross': int}, ....}
def extract_url_dic():
    url='https://www.the-numbers.com/movie/budgets/all/'
    count=1
    name_dic={}
    titles=['Release Date', 'Movie', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']
    host_url='https://www.the-numbers.com'
    while count<5601:
        page_url=url+str(count)
        page_response=requests.get(page_url)
        root=BeautifulSoup(page_response.content.decode('utf-8'), "html.parser")
        
        table=root.table
        tr=table.find_all('tr')[1]
        i=0
        temp_dic={}
        name=None
        for td in tr.find_all('td'):
            if i>0:
                if titles[i-1]=='Movie':
                    name=html.unescape(td.get_text()).replace(u'\xa0', u' ').strip()
                    temp_dic['url']=host_url+td.a.get('href').replace('#tab=summary','')
                elif titles[i-1].startswith('Release'):
                    year=td.get_text().replace(u'\xa0', u' ').split('/')[2]
                    temp_dic[titles[i-1]]=datetime.strptime(html.unescape(td.get_text()).replace(u'\xa0', u' '),'%m/%d/%Y')
                else:
                    try:
                        temp_dic[titles[i-1]]=int(td.get_text().replace(u'\xa0', u' ').replace('$','').replace(',',''))
                    except:
                        temp_dic[titles[i-1]]=html.unescape(td.get_text()).replace(u'\xa0', u' ')
                        pass
            i+=1
            if i==len(titles)+1:
                i=0
                if name!=None:
                    print(name+'('+year+ ')')
                    #print(temp_dic)
                    name_dic[name+'('+year+ ')']=temp_dic
                    name=None
                    temp_dic={}
                    
        count+=100
        #print(len(name_dic))
    
    print('writing...')
    #saved in this pkl
    with open('name_url.pkl', 'wb') as f:
        pickle.dump(name_dic, f)
    
    return name_dic

In [17]:
#There are totally 5517 movies
name_dic=extract_url_dic()
print(len(name_dic))

Avatar(2009)
Pirates of the Caribbean: On Stranger Tides(2011)
Avengers: Age of Ultron(2015)
Star Wars Ep. VII: The Force Awakens(2015)
Pirates of the Caribbean: At World’s End(2007)
Justice League(2017)
Spectre(2015)
The Dark Knight Rises(2012)
The Lone Ranger(2013)
John Carter(2012)
Tangled(2010)
Spider-Man 3(2007)
Captain America: Civil War(2016)
Batman v Superman: Dawn of Justice(2016)
The Hobbit: An Unexpected Journey(2012)
Harry Potter and the Half-Blood Prince(2009)
The Hobbit: The Desolation of Smaug(2013)
The Hobbit: The Battle of the Five Armies(2014)
The Fate of the Furious(2017)
Superman Returns(2006)
Pirates of the Caribbean: Dead Men Tell No Tales(2017)
Quantum of Solace(2008)
The Avengers(2012)
Pirates of the Caribbean: Dead Man’s Chest(2006)
Man of Steel(2013)
The Chronicles of Narnia: Prince Caspian(2008)
The Amazing Spider-Man(2012)
Transformers: The Last Knight(2017)
Jurassic World(2015)
Men in Black 3(2012)
Transformers: Revenge of the Fallen(2009)
Transformers: Age

The Spiderwick Chronicles(2008)
The Incredibles(2004)
A Good Day to Die Hard(2013)
The Alamo(2004)
Cutthroat Island(1995)
The Secret Life of Walter Mitty(2013)
Jumanji: Welcome to the Jungle(2017)
Men in Black(1997)
Toy Story 2(1999)
Rush Hour 2(2001)
Sherlock Holmes(2009)
Ice Age: Dawn of the Dinosaurs(2009)
What Lies Beneath(2000)
Rio(2011)
The Departed(2006)
Charlie's Angels(2000)
Mulan(1998)
Tropic Thunder(2008)
The Girl with the Dragon Tattoo(2011)
Contact(1997)
You Don't Mess With the Zohan(2008)
Die Hard: With a Vengeance(1995)
The Magnificent Seven(2016)
Alvin and the Chipmunks: The Road Chip(2015)
Atlantis: The Lost Empire(2001)
Valkyrie(2008)
Pixels(2015)
Artificial Intelligence: AI(2001)
The Haunted Mansion(2003)
Hollow Man(2000)
The Interpreter(2005)
Percy Jackson: Sea of Monsters(2013)
Lara Croft: Tomb Raider: The Cradle of Life(2003)
Now You See Me 2(2016)
Spy Game(2001)
The Saint(1997)
Mission to Mars(2000)
Bicentennial Man(1999)
Tomb Raider(2018)
King Arthur(2004)
Volca

Sully(2016)
The Bourne Identity(2002)
Couples Retreat(2009)
Goldeneye(1995)
The Italian Job(2003)
Daddy Day Care(2003)
The General's Daughter(1999)
The Prince of Egypt(1998)
Collateral(2004)
Cats & Dogs(2001)
Two Weeks Notice(2002)
Antz(1998)
The Scorpion King(2002)
Red(2010)
Starsky & Hutch(2004)
Days of Thunder(1990)
Cheaper by the Dozen 2(2005)
Eat Pray Love(2010)
Jack Reacher(2012)
Man on Fire(2004)
The Family Man(2000)
Contagion(2011)
Any Given Sunday(1999)
The Horse Whisperer(1998)
Coraline(2009)
Ladder 49(2004)
Deep Blue Sea(1999)
Michael Jackson's This Is It(2009)
Kangaroo Jack(2003)
The Happening(2008)
London Has Fallen(2016)
Rules of Engagement(2000)
The Shaggy Dog(2006)
Jingle All the Way(1996)
Hellboy(2004)
Jack Reacher: Never Go Back(2016)
Baywatch(2017)
A Civil Action(1998)
Joy(2015)
ParaNorman(2012)
The Jackal(1997)
Paycheck(2003)
Up Close & Personal(1996)
The Tale of Despereaux(2008)
The Boxtrolls(2014)
The Dark Tower(2017)
The Tuxedo(2002)
Jack Ryan: Shadow Recruit(201

Training Day(2001)
The Girl on the Train(2016)
Galaxy Quest(1999)
Scary Movie 2(2001)
Blade(1998)
Coach Carter(2005)
The Bounty Hunter(2010)
Changing Lanes(2002)
Anaconda(1997)
Underworld: Evolution(2006)
Coyote Ugly(2000)
Love Actually(2003)
Jack(1996)
The Specialist(1994)
Office Christmas Party(2016)
Tin Cup(1996)
Yours, Mine and Ours(2005)
Kicking and Screaming(2005)
The Hitchhiker's Guide to the Galaxy(2005)
Resident Evil: Extinction(2007)
Fat Albert(2004)
Savages(2012)
The River Wild(1994)
Blended(2014)
All Eyez on Me(2017)
Last Holiday(2006)
16 Blocks(2006)
The Indian in the Cupboard(1995)
Cellular(2004)
The Ant Bully(2006)
Johnny English(2003)
Dune(1984)
Aliens in the Attic(2009)
Across the Universe(2007)
Revolutionary Road(2008)
Babylon A.D.(2008)
Frankenstein(1994)
The Glimmer Man(1996)
Multiplicity(1996)
The Pledge(2001)
The Producers: The Movie Musical(2005)
The Phantom(1996)
Rebound(2005)
The Ghost Writer(2010)
All the Pretty Horses(2000)
Bangkok Dangerous(2008)
Nixon(1995)

House of Wax(2005)
From Hell(2001)
Mirrors(2008)
Queen of the Damned(2002)
Nanny McPhee and the Big Bang(2010)
Untraceable(2008)
Predator 2(1990)
Abduction(2011)
Flash Gordon(1980)
Superhero Movie(2008)
Blast from the Past(1999)
Alex Cross(2012)
Jersey Girl(2004)
Midnight in the Garden of Good and Evil(1997)
Bridget Jones’s Baby(2016)
Heist(2001)
Hoffa(1992)
Whiskey Tango Foxtrot(2016)
Ella Enchanted(2004)
Hitman: Agent 47(2015)
The X-Files: I Want to Believe(2008)
Premium Rush(2012)
Broken City(2013)
Valiant(2005)
Wonder Boys(2000)
Cursed(2005)
Walk Hard: The Dewey Cox Story(2007)
The Four Feathers(2002)
Furry Vengeance(2010)
Parker(2013)
Wimbledon(2004)
Bait(2000)
Dungeons and Dragons(2000)
Lions for Lambs(2007)
Flight of the Intruder(1991)
The Tree of Life(2011)
American Outlaws(2001)
Zoom(2006)
The Shipping News(2001)
The Young Victoria(2009)
Sabotage(2014)
Knock Off(1998)
Whiteout(2009)
Unfinished Business(2015)
The Walk(2015)
The Fountain(2006)
Mandela: Long Walk to Freedom(2013)

...

Black or White(2015)
Down to You(2000)
Harold & Kumar Go to White Castle(2004)
The Contender(2000)
Boiler Room(2000)
Black Christmas(2006)
The Edge of Seventeen(2016)
Jackie(2016)
The Ice Pirates(1984)
Breakin' All the Rules(2004)
Glee: The 3D Concert Movie(2011)
Henry V(1989)
Loving(2016)
The Savages(2007)
Chasing Papi(2003)
The Way of the Gun(2000)
Hamlet 2(2008)
Igby Goes Down(2002)
PCU(1994)
The Ultimate Gift(2007)
Beautiful(2000)
Gracie(2007)
Greater(2016)
Trust the Man(2006)
Undiscovered(2005)
Velvet Goldmine(1998)
Princess Kaiulani(2010)
The Legend of Suriyothai(2003)
Due occhi diabolici(1991)
Dheepan(2016)
La fille du RER(2010)
All or Nothing(2002)
The Red Riding Trilogy: 1974(2010)
Flammen og Citronen(2009)
Leaves of Grass(2010)
Opal Dream(2006)
Skin Trade(2015)
Beneath Hill 60(2014)
Veronika Decides to Die(2015)
Barbarella(1968)
A Haunting in Connecticut 2: The Ghosts of Georgia(2013)
Polisse(2012)
Heist(2015)
Ultramarines(2011)
Crocodile Dundee(1986)
The I Inside(2004)
Awake

Tom yum goong(2006)
Stiff Upper Lips(1999)
The Inbetweeners(2012)
Valhalla Rising(2010)
Bend it Like Beckham(2003)
Crossover(2006)
Sunshine State(2002)
Standing Ovation(2010)
Khiladi 786(2012)
[Rec] 2(2010)
The Sting(1973)
Chariots of Fire(1981)
Diary of a Mad Black Woman(2005)
Shine(1996)
Don Jon(2013)
High Plains Drifter(1973)
Ghost World(2001)
Iris(2001)
Galaxina(1980)
Les Choristes(2004)
Mambo Italiano(2003)
Wonderland(2003)
De battre mon coeur s'est arrêté(2005)
Haevnen(2011)
Le Havre(2011)
Harvard Man(2002)
Salvation Boulevard(2011)
R100(2015)
Miss Julie(2015)
Animals(2008)
Irreversible(2003)
Tao Jie(2013)
Rang De Basanti(2006)
Grabbers(2013)
La Vie d'Adèle – Chapitres 1 & 2(2013)
Hodejegerne(2012)
The Ten(2007)
A Room for Romeo Brass(2000)
Wilson(1944)
Saint Ralph(2005)
Dum Maaro Dum(2011)
Somewhere in Time(1980)
My Big Fat Greek Wedding(2002)
Get Out(2017)
Split(2017)
Paranormal Activity 3(2011)
Saw II(2005)
Insidious Chapter 2(2013)
10 Cloverfield Lane(2016)
Lights Out(2016)
T

Surveillance(2009)
Taxman(1999)
The Kick(2011)
The Betrayed(2009)
The Secret(2006)
2:13(2011)
Batman: The Dark Knight Returns, Part 2(2013)
In the Name of the King III(2014)
Wicked Blood(2014)
Dawn Patrol(2015)
Jagten(2013)
Lords of London(2015)
La marche de l'empereur(2005)
High Anxiety(1977)
Margin Call(2011)
Choke(2008)
L'incomparable mademoiselle C.(2004)
August(2008)
To Write Love On Her Arms(2015)
Carousel(1956)
Whiplash(2014)
Bella(2007)
Cidade de Deus(2003)
Human Traffic(2000)
The Dead Girl(2006)
The Opposite Sex(2015)
A Christmas Story(1983)
Class of 1984(1982)
Entre les murs(2009)
Dreaming of Joseph Lees(1999)
Rosemary's Baby(1968)
The Man Who Shot Liberty Valance(1962)
Maria Full of Grace(2004)
Beginners(2011)
The Meddler(2016)
Adam(2009)
Feast(2006)
It’s a Wonderful Life(1946)
Trainspotting(1996)
Antiviral(2013)
National Lampoon's Animal House(1978)
Paranormal Activity 2(2010)
War Room(2015)
The Purge(2013)
Goldfinger(1964)
Sinister(2012)
The Original Kings of Comedy(2000)


The Lunchbox(2014)
Grace Unplugged(2013)
Happy, Texas(1999)
Saul fia(2015)
My Summer of Love(2005)
Yes(2005)
From Here to Eternity(1953)
You Can't Take It With You(1938)
Foolish(1999)
She Wore a Yellow Ribbon(1949)
The Conversation(1974)
N-Secure(2010)
Sukkar banat(2008)
Out of the Dark(1989)
Bubble(2006)
Dil Jo Bhi Kahey...(2005)
Mississippi Mermaid(1999)
Two Night Stand(2014)
I Love Your Work(2005)
The King’s Thief(1955)
Insidious(2011)
Moonlight(2016)
Cabin Fever(2003)
Waitress(2007)
Kickboxer(1989)
Bloodsport(1988)
Mr. Smith Goes To Washington(1939)
Kids(1995)
The Squid and the Whale(2005)
Kissing Jessica Stein(2002)
Spellbound(1945)
Dawn of the Dead(1979)
Exotica(1994)
The To Do List(2013)
The Gatekeepers(2013)
Buffalo '66(1998)
Repo Man(1984)
I’m Not Ashamed(2016)
Blood Simple(1985)
Nueve Reinas(2002)
The Lords of Salem(2013)
The Ballad of Jack and Rose(2005)
Unsullied(2016)
Killing Zoe(1994)
The Believer(2002)
Snow Angels(2008)
Session 9(2001)
MOOZ-lum(2011)
I Want Someone To Ea

The Journey(2003)
Home(2009)
The Slaughter Rule(2003)
The Living Wake(2010)
Scott Walker: 30 Century Man(2009)
The Californians(2005)
Born to Fly: Elizabeth Streb vs. Gravity(2014)
Detention of the Dead(2013)
Fengkuang de Shitou(2006)
Now is Good(2013)
Everything Put Together(2001)
Drones(2010)
Enter Nowhere(2012)
Girls Gone Dead(2012)
Insomnia Manica(2005)
America Is Still The Place(2014)
The Outrageous Sophie Tucker(2015)
Subconscious(2015)
Roadside(2015)
Diamond Rufff(2014)
Love in the Time of Monsters(2015)
El rey de Najayo(2012)
Fight to the Finish(2014)
Bucky and the Squirrels(2013)
Going Under(1990)
Crossroads(2015)
Alleluia! The Devil's Carnival(2015)
Brooklyn Bizarre(2015)
Crabs!(2016)
The Sound and the Shadow(2014)
Rodeo Girl(2015)
The Little Ponderosa Zoo(2015)
The Toxic Avenger(1984)
Paranormal Activity(2009)
Straight out of Brooklyn(1991)
Brick(2006)
Sunday(1997)
Conversations with Other Women(2006)
Siakh lokhamim: ha'slilim ha'gnouzim(2015)
Poultrygeist: Night of the Chic

In [18]:
#function to get the dictionary of {name:url}
#The result was saved in name_url_bykey.pkl, no need to run it
#https://www.the-numbers.com/movie/budgets/all/5501
#The structure of the dictionary is as following:
#{name of the movie: url(stirng),...}
def extract_url_dic_bykey():
    url='https://www.the-numbers.com/movies/keywords'
    name_dic={}
    host_url='https://www.the-numbers.com'
    page_response=requests.get(url)
    root=BeautifulSoup(page_response.content.decode('utf-8'), "html.parser")
    url_dic=dict()
    for tr in root.find_all('tr'):
        if len(tr.find_all('td'))>0:
            td=tr.find_all('td')[0]
            keyword=html.unescape(td.get_text()).replace(u'\xa0', u' ')
            url_dic[keyword]=host_url+td.a.get('href')
            
    #with open('key_url.pkl', 'wb') as f:
    #    pickle.dump(url_dic, f)

    for keyword,url in url_dic.items():
        subpage_response=requests.get(url)
        subroot=BeautifulSoup(subpage_response.content.decode('utf-8'), "html.parser")
        for tr in subroot.find_all('tr'):
            if len(tr.find_all('td'))>1:
                try:
                    year=html.unescape(tr.find_all('td')[0].get_text()).replace(u'\xa0', u' ').split('/')[2]
                except:
                    print(tr.find_all('td')[0].get_text())
                    continue
                td=tr.find_all('td')[1]
                name=html.unescape(td.get_text()).replace(u'\xa0', u' ')
                name_dic[name+'('+year+')']=host_url+td.a.get('href')
    
    with open('name_url_bykey.pkl', 'wb') as f:
        pickle.dump(name_dic, f)
    
    return name_dic

In [31]:
#get 13458 movies
print(len(extract_url_dic_bykey().keys()))

13458


In [96]:
#get all the movie information and save in movie_info.pkl
#may run several hours
def get_all_movie_info():
    with open('name_url.pkl', 'rb') as f:
        name_dic = pickle.load(f)
    
    total_dic=dict()
    for name in name_dic.keys():
        print(name)
        total_dic[name]=get_info_with_name(name)
    
    with open('movie_info.pkl', 'wb') as f:
        pickle.dump(total_dic, f)
    
    return total_dic

In [103]:
#get all the movie categories information
#may run several hours
def get_all_movie_categories():
    movie_categories=dict()
    keyword, genre, prod, creat, company, country, cast =dict(), dict(), dict(), dict(), dict(), dict(), dict()
    franchise=dict()
    person_name_url=dict()
    
    
    with open('name_url.pkl', 'rb') as f:
        name_dic = pickle.load(f)
    
    for name, value in name_dic.items():
        summary_url=value['url']+"#tab=summary"
        #summary_url=value+"#tab=summary"
        summary_response = requests.get(summary_url, allow_redirects=False)
        if summary_response.status_code != 404:
            summary_dic = parse_page(summary_response.content, "summary")
            #print(summary_dic)
            print(name)
            if 'Genre' in summary_dic.keys():
                genre[summary_dic['Genre']]=genre.get(summary_dic['Genre'],[])+[name]
            if 'Creative Type' in summary_dic.keys():
                creat[summary_dic['Creative Type']]=creat.get(summary_dic['Creative Type'],[])+[name]
            if 'Production Method' in summary_dic.keys():
                for mt in summary_dic['Production Method']:
                    prod[mt]=prod.get(mt,[])+[name]
                    #print(prod.get(mt,[]))
            if 'Production Companies' in summary_dic.keys():
                for cp in summary_dic['Production Companies']:
                    company[cp]=company.get(cp,[])+[name]
            if 'Production Countries' in summary_dic.keys():
                for ct in summary_dic['Production Countries']:
                    country[ct]=country.get(ct,[])+[name]
            if 'Keywords' in summary_dic.keys():
                for keywords in summary_dic['Keywords']:
                    keyword[keywords]=keyword.get(keywords,[])+[name]
            if 'Franchise' in summary_dic.keys():
                franchise[summary_dic['Franchise']]=franchise.get(summary_dic['Franchise'],[])+[name]
                
        cast_url=value['url']+"#tab=cast-and-crew"
        #cast_url=value+"#tab=cast-and-crew"
        cast_response=requests.get(cast_url, allow_redirects=False)
        if cast_response.status_code !=404:
            cast_dic = parse_page(cast_response.content, "cast")
            if 'production' in cast_dic.keys():
                for person in cast_dic['production']:
                    person_name=person['name']
                    if person_name not in person_name_url.keys():
                        person_name_url[person_name]=person['url']
                    temp_dict={'role': person['role'],'name': name}
                    cast[person_name]=cast.get(person_name,[])+[temp_dict]
                    
            if 'leading_members' in cast_dic.keys():
                for person in cast_dic['leading_members']:
                    person_name=person['name']
                    if person_name not in person_name_url.keys():
                        person_name_url[person_name]=person['url']
                        
            if 'supporting_cast' in cast_dic.keys():
                for person in cast_dic['supporting_cast']:
                    person_name=person['name']
                    if person_name not in person_name_url.keys():
                        person_name_url[person_name]=person['url']
            
            if 'cameos' in cast_dic.keys():
                for person in cast_dic['cameos']:
                    person_name=person['name']
                    if person_name not in person_name_url.keys():
                        person_name_url[person_name]=person['url']
                        
            if 'uncategorized' in cast_dic.keys():
                for person in cast_dic['uncategorized']:
                    person_name=person['name']
                    if person_name not in person_name_url.keys():
                        person_name_url[person_name]=person['url']
                        
        #pprint(keyword)
        #pprint(genre)
        #pprint(prod)
        #pprint(creat)
        #pprint(company)
        #pprint(country)
        #pprint(cast)
        #pprint(franchise)
        
    movie_categories['Genre']=genre
    movie_categories['Creative Type']=creat
    movie_categories['Production Method']=prod
    movie_categories['Production Companies']=company
    movie_categories['Production Countries']=country
    movie_categories['Keywords']=keyword
    movie_categories['Franchise']=franchise
    movie_categories['cast']=cast
    
    print('Writing...')
    with open('movie_categories.pkl', 'wb') as f:
        pickle.dump(movie_categories, f)
        
    with open('person_name_url.pkl', 'wb') as f:
        pickle.dump(person_name_url, f)
        
    return movie_categories

In [105]:
print(len(get_all_movie_categories()))

Avatar(2009)
Pirates of the Caribbean: On Stranger Tides(2011)
Avengers: Age of Ultron(2015)
Star Wars Ep. VII: The Force Awakens(2015)
Pirates of the Caribbean: At World’s End(2007)
Justice League(2017)
Spectre(2015)
The Dark Knight Rises(2012)
The Lone Ranger(2013)
John Carter(2012)
Tangled(2010)
Spider-Man 3(2007)
Captain America: Civil War(2016)
Batman v Superman: Dawn of Justice(2016)
The Hobbit: An Unexpected Journey(2012)
Harry Potter and the Half-Blood Prince(2009)
The Hobbit: The Desolation of Smaug(2013)
The Hobbit: The Battle of the Five Armies(2014)
The Fate of the Furious(2017)
Superman Returns(2006)
Pirates of the Caribbean: Dead Men Tell No Tales(2017)
Quantum of Solace(2008)
The Avengers(2012)
Pirates of the Caribbean: Dead Man’s Chest(2006)
Man of Steel(2013)
The Chronicles of Narnia: Prince Caspian(2008)
The Amazing Spider-Man(2012)
Transformers: The Last Knight(2017)
Jurassic World(2015)
Men in Black 3(2012)
Transformers: Revenge of the Fallen(2009)
Transformers: Age

I, Robot(2004)
Stuart Little(1999)
The Princess and the Frog(2009)
10,000 B.C.(2008)
Ice Age: Collision Course(2016)
Town & Country(2001)
Kingsman: The Golden Circle(2017)
Gone in 60 Seconds(2000)
Gladiator(2000)
The Hangover 3(2013)
A Wrinkle in Time(2018)
Public Enemies(2009)
Casino Royale(2006)
Minority Report(2002)
Cloud Atlas(2012)
Harry Potter and the Chamber of Secrets(2002)
Terminator 2: Judgment Day(1991)
Batman Forever(1995)
Planet of the Apes(2001)
National Treasure(2004)
Django Unchained(2012)
Little Fockers(2010)
True Lies(1994)
American Gangster(2007)
Cloudy with a Chance of Meatballs(2009)
The Other Guys(2010)
Lemony Snicket's A Series of Unfortunate Events(2004)
The Wolf of Wall Street(2013)
Epic(2013)
Eraser(1996)
The Hunchback of Notre Dame(1996)
The Emperor's New Groove(2000)
The Expendables 2(2012)
Where the Wild Things Are(2009)
Eragon(2006)
Hercules(2014)
The Tourist(2010)
End of Days(1999)
The Stepford Wives(2004)
Surf's Up(2007)
Blood Diamond(2006)
Legend of the

The Dilemma(2011)
Rumor Has It(2005)
The Siege(1998)
Stardust(2007)
Rock of Ages(2012)
Seven Years in Tibet(1997)
Sky Captain and the World of Tomorrow(2004)
I Spy(2002)
Bad Company(2002)
Doom(2005)
The Finest Hours(2016)
Escape Plan(2013)
Killer Elite(2011)
Hard Rain(1998)
Hart's War(2002)
Rollerball(2002)
The Legend of Hercules(2014)
Ballistic: Ecks vs. Sever(2002)
Osmosis Jones(2001)
Legends of Oz: Dorothy’s Return(2014)
Blackhat(2015)
Basic Instinct 2(2006)
Agora(2010)
Despicable Me(2010)
Dinner for Schmucks(2010)
Red Sparrow(2018)
The Twilight Saga: Eclipse(2010)
The Sum of All Fears(2002)
Ted 2(2015)
The Score(2001)
Hall Pass(2011)
Money Train(1995)
Dreamcatcher(2003)
Mystery Men(1999)
The Insider(1999)
Downsizing(2017)
Doctor Seuss' The Lorax(2012)
Body of Lies(2008)
Abraham Lincoln: Vampire Hunter(2012)
Last Man Standing(1996)
The Last Legion(2007)
Entrapment(1999)
The X Files: Fight the Future(1998)
Need for Speed(2014)
Saving Private Ryan(1998)
What Women Want(2000)
Lincoln(2

The Forgotten(2004)
Herbie: Fully Loaded(2005)
Shanghai Knights(2003)
Planes: Fire and Rescue(2014)
Curious George(2006)
American Reunion(2012)
Hansel & Gretel: Witch Hunters(2013)
I am Number Four(2011)
Don’t Say a Word(2001)
13 Hours: The Secret Soldiers of Benghazi(2016)
Unfaithful(2002)
American Made(2017)
Syriana(2005)
Resident Evil: Apocalypse(2004)
The Book of Life(2014)
Absolute Power(1997)
Firewall(2006)
The Game(1997)
G.I. Jane(1997)
Into the Storm(2014)
The Judge(2014)
Silent Hill(2006)
The Replacements(2000)
The Negotiator(1998)
War Dogs(2016)
Beverly Hills Cop III(1994)
Gremlins 2: The New Batch(1990)
The Peacemaker(1997)
Out of Time(2003)
Bridget Jones: The Edge Of Reason(2004)
The Beach(2000)
The Adventures of Sharkboy and Lavagirl in 3-D(2005)
On Deadly Ground(1994)
Ninja Assassin(2009)
Raising Helen(2004)
The Nice Guys(2016)
For Love of the Game(1999)
Jack Frost(1998)
Marmaduke(2010)
Striptease(1996)
Hereafter(2010)
Murder by Numbers(2002)
Assassins(1995)
Zoolander 2(2

Duplex(2003)
Paranoia(2013)
Raise the Titanic(1980)
Baahubali: The Beginning(2015)
Impostor(2002)
The Hurricane Heist(2018)
Victor Frankenstein(2015)
Extreme Ops(2002)
Just Visiting(2001)
Snowpiercer(2014)
Sunshine(2007)
The Hudsucker Proxy(1994)
Billy Lynn’s Long Halftime Walk(2016)
Pirates(1986)
Delgo(2008)
The Hunting Party(2007)
Alex Rider: Operation Stormbreaker(2006)
Red Cliff(2009)
The Last Shot(2004)
The Warlords(2010)
Nomad(2007)
Un monstre à Paris(2013)
USS Indianapolis: Men of Courage(2016)
Dragon Nest Warriors' Dawn(2014)
The Crow(2019)
The Time Traveler's Wife(2009)
Superman III(1983)
Because I Said So(2007)
Frankenweenie(2012)
Sgt. Bilko(1996)
Serenity(2005)
Against the Ropes(2004)
Yi dai zong shi(2013)
The Fast and the Furious(2001)
Sweet Home Alabama(2002)
The Ugly Truth(2009)
Spy Kids 2: The Island of Lost Dreams(2002)
Star Trek: Generations(1994)
Captain Underpants: The First Epic Movie(2017)
Paul Blart: Mall Cop 2(2015)
Mouse Hunt(1997)
Why Him?(2016)
Water for Eleph

...

Lights Out(2016)
The Visit(2015)
Jackass: The Movie(2002)
Happy Death Day(2017)
Paranormal Activity 4(2012)
Ouija(2014)
A Nightmare On Elm Street 3: Dream Warriors(1987)
No se Aceptan Devoluciones(2013)
The Gift(2015)
The Big Sick(2017)
The Return of the Pink Panther(1975)
The Robe(1953)
Freddy’s Dead: The Final Nightmare(1991)
Monster(2003)
Paranormal Activity: The Marked Ones(2014)
20,000 Leagues Under the Sea(1954)
Oculus(2014)
Dallas Buyers Club(2013)
No Escape(2015)
The Elephant Man(1980)
The Lazarus Effect(2015)
Memento(2001)
Our Idiot Brother(2011)
Clerks II(2006)
The Players Club(1998)
Billy Elliot(2000)
The Way Way Back(2013)
As Above, So Below(2014)
God’s Not Dead 2(2016)
The Apostle(1997)
The Man From Snowy River(1982)
House Party 2(1991)
Doug's 1st Movie(1999)
Race(2016)
Mommie Dearest(1981)
Still Alice(2015)
Halloween 4: The Return of Michael Myers(1988)
Apollo 18(2011)
Addicted(2014)
Paul, Apostle of Christ(2018)
O (Othello)(2001)
Halloween: The Curse of Michael Myers(199

The Dead Girl(2006)
The Opposite Sex(2015)
A Christmas Story(1983)
Class of 1984(1982)
Entre les murs(2009)
Dreaming of Joseph Lees(1999)
Rosemary's Baby(1968)
The Man Who Shot Liberty Valance(1962)
Maria Full of Grace(2004)
Beginners(2011)
The Meddler(2016)
Adam(2009)
Feast(2006)
It’s a Wonderful Life(1946)
Trainspotting(1996)
Antiviral(2013)
National Lampoon's Animal House(1978)
Paranormal Activity 2(2010)
War Room(2015)
The Purge(2013)
Goldfinger(1964)
Sinister(2012)
The Original Kings of Comedy(2000)
The Bridge on the River Kwai(1957)
Coming Home(1978)
Waking Ned Devine(1998)
Air Bud(1997)
Bowling for Columbine(2002)
A Nightmare on Elm Street Part 2: Freddy's Revenge(1985)
A Room with a View(1986)
Love and Death(1975)
Friday the 13th Part VI: Jason Lives(1986)
Martin Lawrence Live: RunTelDat(2002)
The Apartment(1960)
Cat on a Hot Tin Roof(1958)
Beneath the Planet of the Apes(1970)
Pokemon 3: The Movie(2001)
Jason Goes to Hell: The Final Friday(1993)
Spaced Invaders(1990)
Family Plo

POM Wonderful Presents: The Greatest Movie Ever Sold(2011)
Travellers and Magicians(2005)
Ed and his Dead Mother(1993)
Deadline(2009)
Sublime(2007)
Dysfunctional Friends(2012)
Independence Daysaster(2013)
Against the Wild(2014)
Beginners Guide to Sex(2015)
Lola rennt(1999)
May(2003)
I Spit on Your Grave(2010)
Living Dark: The Story of Ted the Caver(2013)
In the Bedroom(2001)
La misma luna(2008)
Conquest of the Planet of the Apes(1972)
The Lunchbox(2014)
Grace Unplugged(2013)
Happy, Texas(1999)
Saul fia(2015)
My Summer of Love(2005)
Yes(2005)
From Here to Eternity(1953)
You Can't Take It With You(1938)
Foolish(1999)
She Wore a Yellow Ribbon(1949)
The Conversation(1974)
N-Secure(2010)
Sukkar banat(2008)
Out of the Dark(1989)
Bubble(2006)
Dil Jo Bhi Kahey...(2005)
Mississippi Mermaid(1999)
Two Night Stand(2014)
I Love Your Work(2005)
The King’s Thief(1955)
Insidious(2011)
Moonlight(2016)
Cabin Fever(2003)
Waitress(2007)
Kickboxer(1989)
Bloodsport(1988)
Mr. Smith Goes To Washington(1939)
K

The Blair Witch Project(1999)
Woodstock(1970)
The Kentucky Fried Movie(1977)
Per qualche dollaro in più(1967)
Mercy Streets(2000)
Water & Power(2014)
Arnolds Park(2007)
Broken Vessels(1999)
Drunk Wedding(2015)
They Will Have to Kill Us First(2016)
Crop Circles: Quest for Truth(2002)
The Maid's Room(2014)
Light from the Darkroom(2014)
Irreplaceable(2015)
A Hard Day's Night(1964)
The Harvest/La Cosecha(2011)
Julija in alfa Romeo(2015)
Mata Hari(1931)
Friday the 13th(1980)
Love Letters(1984)
Faith Connections(2014)
High Road(2012)
Fireproof(2008)
Benji(1974)
Open Water(2004)
Kingdom of the Spiders(1977)
Mad Hot Ballroom(2005)
The Station Agent(2003)
To Save a Life(2010)
Wordplay(2006)
Beyond the Mat(1999)
The Singles Ward(2002)
Sholem Aleichem: Laughing in the Darkness(2011)
Osama(2004)
Groove(2000)
The R.M.(2003)
Twin Falls Idaho(1999)
Mean Creek(2004)
Drinking Buddies(2013)
Hurricane Streets(1998)
Never Again(2002)
Shichinin no samurai(1956)
Civil Brand(2003)
Monsters(2010)
Lonesome Jim

El Mariachi(1993)
Primer(2004)
Cavite(2006)
The Mongol King(2004)
Following(1999)
Return to the Land of Wonders(2005)
Signed Sealed Delivered(2015)
A Plague So Pleasant(2015)
My Date With Drew(2005)
Writing...
8


In [49]:
def parse_person_page(html_page, title):
    root=BeautifulSoup(html_page.decode('utf-8'), "html.parser")
    lis=dict()
    if title=="summary":
        title_name=['', 'role', 'Movies', 'Domestic Box Office',  'International Box Office', 'Worldwide Box Office']
        lis['Career Summary']=center_table_helper(root, title_name, 'h2', 'Career Summary', 'person')
        
        title_name=['Record', 'Rank', 'Amount']
        #get the rank list
        lis['Latest Ranking']=center_table_helper(root, title_name, 'h2', 'Latest Ranking on Selected Box Office Record Lists')
        pass      
    
    elif title=="acting":
        title_name=['Release Date', 'Title', 'Role' ,' Domestic Box Office', 'International Box Office', 'Worldwide Box Office']
        lis['All Acting Credits']= center_table_helper(root, title_name, 'h2','All Acting Credits', 'person')
        
        title_name=['Release Date', 'Title', 'Opening Weekend Box Office', 'Max Theater Count', 'Domestic Box Office', 'Worldwide Box Office', 'Domestic Share']
        lis['Leading or Lead Ensemble Roles']=center_table_helper(root, title_name, 'h2', 'Leading or Lead Ensemble Roles','person')
        
        title_name=['Release Date', 'Title' ,'Opening Weekend Box Office', ' Max Theater Count', 'Domestic Box Office', 'Worldwide Box Office', 'Domestic Share']
        lis['Supporting Roles']=center_table_helper(root, title_name, 'h2', 'Supporting Roles', 'person')
        
        title_name=['Record', 'Rank', 'Amount']
        lis['Latest Ranking']=center_table_helper(root, title_name, 'h2', 'Latest Ranking on All Acting Box Office Record Lists', 'person')
        pass

    
    
    elif title=="technical":
        title_name=['Release Date', 'Title', 'Role' ,' Domestic Box Office', 'International Box Office', 'Worldwide Box Office']
        lis['All Technical Credits']= center_table_helper(root, title_name, 'h2','All Technical Credits', 'person')
        
        title_name=['Release Date', 'Title', 'Opening Weekend Box Office', 'Max Theater Count', 'Domestic Box Office', 'Worldwide Box Office', 'Domestic Share']
        lis['Director Credits']= center_table_helper(root, title_name, 'h2','Director Credits', 'person')
        
        title_name=['Release Date', 'Title', 'Opening Weekend Box Office', 'Max Theater Count', 'Domestic Box Office', 'Worldwide Box Office', 'Domestic Share']
        lis['Writer Credits']= center_table_helper(root, title_name, 'h2','Writer Credits', 'person')
        
        title_name=['Record', 'Rank', 'Amount']
        lis['Latest Ranking']= center_table_helper(root, title_name, 'h2','Latest Ranking on All Technical Box Office Record Lists','person')
        pass
    
    elif title=="news":
        host_url="https://www.the-numbers.com"
        url_list=list()
        for item in root.find_all('a'):
            if item.get_text().replace(u'\xa0', u' ')=='More...':
                url_list.append(host_url+item.get('href'))
                
        text_list=list()
        for url in url_list:
            url_response=requests.get(url)
            url_html=url_response.content
            url_root=BeautifulSoup(url_html.decode('utf-8'), "html.parser")
            for div in url_root.find_all('div'):
                if div.get('id')=='news':
                    news_content=html.unescape(div.get_text()).replace(u'\xa0', u' ').replace('\n',' ').replace('\r','')
                    text_list.append(news_content)
                    break

        lis['news']=text_list
        pass
    
    return lis

## PKL Explanation
Currentlt, there are 4 pkl files.
In name_url.pkl, there is a dictionary with the structure {name of movie: {'url': string, 'Release Date': datetime, 'Production Budget': int, 'Domestic Gross': int, 'Worldwide Gross': int},...}, where th key is the the movie name(year). Currently there are 5517 movies.
In name_url_bykey.pkl, there is a dictionaey with the structure {movie name: url,...}, where the key is the movie name(year), the value is the url link to the page contains the information of the movie. Currently, there are 13458 movies, but not used. 
In person_name_url.pkl, there is a dictionary with the structure {person name: url,...}, where the key is the user name and the value is the url link to the page contains the inforation of the user.
In movie_categories.pkl, there is a dictionary with the movies name lists categoried with different tags. The details is explained in the api function get_movie_categories().

# The following is the api functions

# First API

In [35]:
#get the movie information with the movie name(movie name+(year))
#The structure of the output is as following:
#   {'box_office': {'daliy': [{'Change':float, 
#                              'Date': datetime, 
#                              'Gross': int, 
#                              'Per Theater':int, 
#                              'Rank':int, 
#                              'Theaters':int,
#                              'Total Gross': int,
#                              'Week':int},... ],   
#                 'demostic': [{'Rank': int,
#                              'Record': string,
#                              'Revenue': int},...],
#                 'weekend': [{'Change': float,
#                             'Date': datetime,
#                             'Gross': int,
#                             'Per Theater': int,
#                             'Rank': int,
#                             'Theaters': int,
#                             'Total Gross': int,
#                             'Week': int},...],
#                 'weekly': [{'Change': float,
#                            'Date': datetime,
#                            'Gross': int,
#                            'Per Theater': int,
#                            'Rank': int,
#                            'Theaters': int,
#                            'Total Gross': int,
#                            'Week': int},...]},
#   'cast': {'cameos': [{'name': string,
#                      'url': string},...],
#            'leading_members': [{'name': string,
#                               'url': string},...],
#            'production': [{'name': string,
#                          'role': string,
#                          'url': string},...],
#            'supporting_cast': [{'name': string,
#                               'url': string},...],
#            'uncategorized': [{'name': string,
#                             'url': string},...]},
#   'international': {'Box Office Summary': [{'Maximum Theaters': int,
#                                           'Opening Weekend': int,
#                                           'Opening Weekend Theaters': int,
#                                           'Release Date': datetime,
#                                           'Report Date': datetime,
#                                           'Territory': sting,
#                                           'Theatrical Engagements': int,
#                                           'Total Box Office': int},...],
#                     'International Box Office': [{'Rank': int,
#                                                 'Record': string,
#                                                 'Revenue': int},...],
#                     'Worldwide Box Office': [{'Rank': int,
#                                             'Record': string,
#                                             'Revenue': int},...]},
#    'news': {'news': [string, ....]},
#    'summary': {'Budget': int,
#             'Creative Type': string,
#             'Domestic Releases': {'IMAX': datetime,        (may not have IMAX version)
#                                   'Wide': datetime},
#             'Franchise': string,
#             'Genre': string,
#             'International Releases': {location: {'Wide': datetime},...},   (may have IMAX version)
#             'Keywords':[string,...],
#             'MPAA Rating': string,
#             'Production Companies': [string,...],
#             'Production Countries': [string,...],
#             'Production Method': [string,...],
#             'Running Time': string,
#             'Source': string,
#             'Video Release': string,
#             'ranking on other records': [{'Amount': int,
#                                           'Chart Date': string,
#                                           'Days In Release': int,
#                                           'Rank': int,
#                                           'Record': string},...]}                 
#}
def get_info_with_name(name):
    with open('name_url_bykey.pkl', 'rb') as f:
        name_dic = pickle.load(f)
    
    name_info=get_info(name_dic[name])
    
    return name_info

In [36]:
#Example 1
pprint(get_info_with_name('Black Panther(2018)'))

{'box_office': {'daliy': [{'Change': 0.0,
                           'Date': datetime.datetime(2018, 2, 16, 0, 0),
                           'Gross': 75941146,
                           'Per Theater': 18891,
                           'Rank': 1,
                           'Theaters': 4020,
                           'Total Gross': 75941146,
                           'Week': 1},
                          {'Change': -0.13,
                           'Date': datetime.datetime(2018, 2, 17, 0, 0),
                           'Gross': 65995366,
                           'Per Theater': 16417,
                           'Rank': 1,
                           'Theaters': 4020,
                           'Total Gross': 141936512,
                           'Week': 2},
                          {'Change': -0.09,
                           'Date': datetime.datetime(2018, 2, 18, 0, 0),
                           'Gross': 60067439,
                           'Per Theater': 14942,
                 

                            'Per Theater': 72625,
                            'Rank': 1,
                            'Theaters': 4020,
                            'Total Gross': 291954422,
                            'Week': 1},
                           {'Change': -0.51,
                            'Date': datetime.datetime(2018, 2, 23, 0, 0),
                            'Gross': 143445615,
                            'Per Theater': 35683,
                            'Rank': 1,
                            'Theaters': 4020,
                            'Total Gross': 435400037,
                            'Week': 2},
                           {'Change': -0.4,
                            'Date': datetime.datetime(2018, 3, 2, 0, 0),
                            'Gross': 85479564,
                            'Per Theater': 20930,
                            'Rank': 1,
                            'Theaters': 4084,
                            'Total Gross': 520879601,
                      

                                                 'Record': 'All Time '
                                                           'International Box '
                                                           'Office for Super '
                                                           'Hero Movies',
                                                 'Revenue': 642073119},
                                                {'Rank': 13,
                                                 'Record': 'All Time '
                                                           'International Box '
                                                           'Office for Action '
                                                           'Movies',
                                                 'Revenue': 642073119},
                                                {'Rank': 27,
                                                 'Record': 'All Time '
                                                           

                   'get to $100 million domestically.     - Weekend Box Office '
                   'Chart    - Pacific Rim: Uprising Comparisons  - Sherlock '
                   'Gnomes Comparisons  - Paul, Apostle of Christ Comparisons  '
                   '- Midnight Sun Comparisons  - Unsane Comparisons   '
                   'C.S.Strowbridge ',
                   '  Weekend Estimates: Pacific Rim Leads Weak Field of '
                   'Openers March 25th, 2018        Black Panther’s run at the '
                   'top of the domestic chart will finally come to an end this '
                   'weekend, with Pacific Rim: Uprising taking over as the '
                   'most popular film in the US and Canada. The switch over is '
                   'mostly due to Panther beginning to run out of steam, '
                   'rather than Uprising showing any particular strength '
                   'though. Universal’s sci-fi action movie will pick up '
                   '$28.005

                   'million, while Get Out held on better than expected with '
                   '$28.24 million. There’s little chance 2018 will be able to '
                   'beat that at the top, so 2018’s winning streak might come '
                   'to an end.     Black Panther opened faster than expected '
                   'and held on better than most monster hits. It has already '
                   'reached $400 million domestically and will likely nearly '
                   'get to $500 million over the weekend. Most are expecting '
                   'it to earn just short of $60 million during the weekend, '
                   'which will be more than the rest of the top five will make '
                   'combined.   Red Sparrow has been a divisive film with the '
                   'reviews at almost exactly 50% positive. A lot of critics '
                   'are praising Jennifer Lawrence’s performances, but even '
                   'critics that love her in t

                   'overly bullish about its box office chances. Finally, '
                   'there’s Every Day, a Young Adult adaptation. This '
                   'particular genre is off its peak, so most assume this '
                   'won’t be a major hit. This weekend last year, Get Out '
                   'debuted with $33.38 million. None of the new releases will '
                   'match that this year, but Black Panther will lead 2018 to '
                   'victory in the year-over-year comparison.     '
                   'Annihilation     Trailer: Click to Play Official Site: '
                   'ForThoseThatFollow.com Distributor: Paramount Pictures '
                   'Release Date: February 23rd, 2018 MPAA Rating: R for '
                   'violence, bloody images, language and some sexuality.  '
                   'Source: Based on Fiction Book/Short Story Genre: '
                   'Thriller/Suspense Keywords:  Unnamed Character, '
                   'Envi

In [37]:
#Example 2
pprint(get_info_with_name('Quantum of Solace(2008)'))

{'box_office': {'daliy': [{'Change': 0.0,
                           'Date': datetime.datetime(2008, 11, 14, 0, 0),
                           'Gross': 27007026,
                           'Per Theater': 7826,
                           'Rank': 1,
                           'Theaters': 3451,
                           'Total Gross': 27007026,
                           'Week': 1},
                          {'Change': -0.04,
                           'Date': datetime.datetime(2008, 11, 15, 0, 0),
                           'Gross': 25831827,
                           'Per Theater': 7485,
                           'Rank': 1,
                           'Theaters': 3451,
                           'Total Gross': 52838853,
                           'Week': 2},
                          {'Change': -0.43,
                           'Date': datetime.datetime(2008, 11, 16, 0, 0),
                           'Gross': 14690029,
                           'Per Theater': 4257,
                  

                   '$119.90 million in sales from 6.67 million units sold, '
                   'which is the best sales 2009 has produced so far. Bolt '
                   'slipped to fourth place but held on better selling 625,000 '
                   'units and $11.19 million for totals of $37.21 million and '
                   '2.11 million units. Opening in fifth place was Seven '
                   'Pounds, which was the only new release that charted this '
                   "week that I didn't review. It pulled in $8.64 million "
                   'after selling 506,000 units, which is inline with '
                   'expectations.   Bedtime Stories opened, sort of, scoring '
                   'seventh place with 221,000 units and $3.77 million in '
                   'sales, but that was over just a couple of days. The final '
                   'new release to chart was Hannah Montana - Keeping It Real, '
                   'which placed 23rd with a hair under 40,000 unit

                   'could still hit $200 million worldwide.   Italians slipped '
                   'to second place in Italy and fell to 26th place '
                   'internationally with $1.95 million on 437 screens over the '
                   'weekend for a total of $15.68 million after three.  '
                   'Madagascar: Escape 2 Africa nearly fell out of the top 30 '
                   'with $1.77 million on 2312 screens in 36 markets for a '
                   'total of $398.66 million internationally and $578.26 '
                   'million worldwide.   Role Models added $1.68 million on '
                   '858 screens in 17 markets for a total of $17.55 million. '
                   'This includes an opening in France, but the film bombed in '
                   'that market earning just $196,000 on 110 screens.   Look '
                   'for a Star evaporated this week down 68% to $1.52 million '
                   'on 331 screens in 3 markets for a total of $1

                   'it will likely be a big seller when it hits the home '
                   'market this time next year.   Twilight came in second, or '
                   'third; it depends on whether you are looking at the 3-day '
                   'or the 5-day weekend. Over three days the film earned '
                   '$26.34 million, which was down more than 62%. Even given '
                   'the Fanboy Effect, this was a steep decline given the '
                   'holiday. That said, the film already had $119.71 million '
                   'and should reach $150 million domestically, which is more '
                   'than enough to pay for its combined budget. (In fact, it '
                   "might have done so already depending on the studio's share "
                   'of the box office, which tends to be front loaded.) This '
                   'is the first major hit for Summit Entertainment, but '
                   'hopefully they can turn the franchise int

                   'movies see our Coming Soon section.  In case you have any '
                   'corrections or additional US release information, please '
                   'post them at our forum at  Release Date Changes   Pim '
                   'Beers Source: Variety, Exhibitor Relations and The '
                   'Hollywood Stock Exchange  Filed under: Release Date '
                   'Updates, Quantum of Solace, The Curious Case of Benjamin '
                   'Button, The International, My Life in Ruins, Sex Drive, '
                   'Bandslam, Hamlet 2, Flash of Genius, The Brothers Bloom, '
                   'Delgo, The Other End of the Line, Walking on Dead Fish, '
                   'Fame, Sci-Fi Movie, The Cabin in the Woods, Youth in '
                   'Revolt, Smother, The Hobbit: The Desolation of Smaug, The '
                   'Hobbit: An Unexpected Journey, Appaloosa, Amelia  ',
                   '  Cast Updates - January 3, 2008 January 3rd, 2008    Th

# Second API

In [38]:
#get all the list of categories
#The structure of the output is as following:
#{ 'Genre': {keyword(genre name): [string(movie name),...],...},
#  'Creative Type': {keyword(type name): [string(movie name),...],...},
#  'Production Method':{keyword(method name): [string(movie name),...],...},
#  'Production Companies':{keyword(company name): [string(movie name),...],...},
#  'Production Countries':{keyword(country name): [string(movie name),...],...},
#  'Franchise':{keyword(franchise name): [string(movie name),...],...},
#  'Keywords':{keyword(keyword name): [string(movie name),...],...},
#  'cast':{keyword(actor name): [string(movie name),...],...},
#}
def get_movie_categories():
    with open('movie_categories.pkl', 'rb') as f:
        test_categories = pickle.load(f)
    
    return test_categories

In [39]:
#Example
test_categories=get_movie_categories()
#pprint(test_categories)
print(len(test_categories['Keywords'].keys()))

1777


In [40]:
#Example
print(test_categories['Genre'].keys())

dict_keys(['Action', 'Adventure', 'Western', 'Musical', 'Thriller/Suspense', 'Drama', 'Comedy', 'Horror', 'Black Comedy', 'Romantic Comedy', 'Documentary', 'Concert/Performance'])


In [41]:
#Example
print(test_categories['Creative Type'].keys())

dict_keys(['Science Fiction', 'Fantasy', 'Super Hero', 'Historical Fiction', 'Contemporary Fiction', 'Kids Fiction', 'Dramatization', 'Multiple Creative Types', 'Factual'])


In [42]:
#Example
print(test_categories['Production Method'].keys())

dict_keys(['Animation', 'Live Action', 'Digital Animation', 'Hand Animation', 'Stop-Motion Animation', 'Rotoscoping', 'Multiple Production Methods'])


In [43]:
#Example
print(test_categories['Production Companies'].keys())

dict_keys(['Dune Entertainment', ' 20th Century Fox', ' Ingenious Film Partners', 'Walt Disney Pictures', 'Marvel Studios', 'Lucasfilm', ' Bad Robot', ' Jerry Bruckheimer', 'DC Films', ' RatPac Entertainment', ' Atlas Entertainment', ' Cruel and Unusual Films', 'Eon Productions', 'Warner Bros.', ' Legendary Pictures', ' Syncopy', ' Blind Wink', ' Infinitum Nihil', ' Jim Morris', 'Walt Disney Animation Studios', 'Columbia Pictures', ' Marvel Studios', ' Laura Ziskin Productions', ' Dune Entertainment', 'New Line Cinema', ' Metro-Goldwyn-Mayer Pictures', ' Wingnut Films', 'Heyday Films', 'Wingnut Films', 'Universal Pictures', ' Original Film', ' One Race Films', ' Jon Peters', ' Bad Hat Harry Productions', ' Jerry Bruckheimer Films', ' Paramount Pictures', ' Walden Media', ' Mark Johnson', ' Silverbell Films', 'Hasbro Studios', ' di Bonaventura Pictures', ' Amblin Entertainment', ' Hemisphere Media Capital', ' Parkes+Macdonald Productions', ' Image Nation', 'DreamWorks Pictures', ' Hasbr

In [44]:
#Example
print(test_categories['Production Countries'].keys())

dict_keys(['United States', 'United Kingdom', 'New Zealand', 'France', 'Canada', 'Germany', 'Australia', 'Netherlands', 'Malta', 'China', 'Spain', 'Dominican Republic', 'Paraguay', 'Uruguay', 'Hungary', 'Japan', 'Kuwait', 'Switzerland', 'Italy', 'Morocco', 'Namibia', 'Finland', 'Czech Republic', 'Denmark', 'Argentina', 'Mexico', 'Hong Kong', 'United Arab Emirates', 'Slovakia', 'Bahamas', 'Panama', 'Iceland', 'Republic of Korea', 'Brazil', 'Ireland', 'India', 'Bosnia and Herzegovina', 'Croatia', 'Kazakhstan', 'Russian Federation', 'Tunisia', 'South Africa', 'Libyan Arab Jamahiriya', 'Belgium', 'Saudi Arabia', 'Mongolia', 'Luxembourg', 'Ukraine', 'Malaysia', 'Chile', 'Poland', 'Romania', 'Turkey', 'Taiwan', ' Province of China', 'Algeria', 'Bulgaria', 'Austria', 'Norway', 'Sweden', 'Thailand', 'Georgia', 'Nigeria', 'Cyprus', 'Jordan', 'Greece', 'Cambodia', 'Israel', 'Colombia', 'Cayman Islands', 'Lebanon', 'Kyrgyzstan', 'Peru', 'Indonesia', 'Pakistan', 'Puerto Rico', 'Islamic Republic of

In [45]:
#Example
print(test_categories['Keywords'].keys())

dict_keys(['3-D', ' Invented Language', ' Visual Effects', ' Alien Invasion', ' IMAX: DMR', ' Humans as Aliens', ' Language Barrier', ' 2010 Oscars Best Picture Nominee', 'Pirates', ' Zombies', ' Romance', ' Epilogue', ' Revenge', ' Sequels Without Their Original Stars', ' 3-D', ' Foreign Language', ' Escape', ' Mermaids', 'Marvel Comics', ' Robot', ' End of the World', ' Hallucinations', 'Space Opera', ' Good vs. Evil', ' Delayed Sequel', ' Intertitle', ' Sequels With Returning Stars', ' Dysfunctional Family', ' Gratuitous Cameos', 'Ensemble', ' D.C. Comics', ' Cyborg', ' Secret Identity', ' Surprise Twist', ' Scene in End Credits', ' Slow Motion Scene', ' Underwater', ' Atlantis', ' Kidnap', ' Royalty', 'Secret Agent', ' Shadowy Organization', ' The Past Catching up with You', ' Development Hell', ' Prologue', ' Mexico', 'Vigilante', ' Terrorism', ' Fugitive / On the Run', ' Non-Chronological', 'Native Americans', ' Government Corruption', ' Genocide', ' Breaking the Fourth Wall', ' 

# Third API

In [50]:
#given the name of the actor/writor/director/..., return the information dictionary
#The structure of the output is as following:
#{'acting': {'All Acting Credits': [{' Domestic Box Office': int,
#                                    'International Box Office': int,
#                                    'Release Date': datetime,
#                                    'Role': string,
#                                    'Title': string,
#                                    'Worldwide Box Office': int},...],
#            'Latest Ranking': [{'Amount': int,
#                                'Rank': int,
#                                'Record': string},...],
#            'Leading or Lead Ensemble Roles': [{'Domestic Box Office': int,
#                                                'Domestic Share': string,
#                                                'Max Theater Count': int,
#                                                'Opening Weekend Box Office': int,
#                                                'Release Date': datetime,
#                                                'Title': string,
#                                                'Worldwide Box Office': int},...],
#             'Supporting Roles': [{' Max Theater Count': int,
#                                  'Domestic Box Office': int,
#                                  'Domestic Share': float,
#                                  'Opening Weekend Box Office': int,
#                                  'Release Date': datetime,
#                                  'Title': string,
#                                  'Worldwide Box Office': int},...]}
#  'news': {'news': [string, ...]}
#  'summary': {'Career Summary': [{'Domestic Box Office': int,
#                                'International Box Office': int,
#                                 'Movies': int,
#                                 'Worldwide Box Office': int,
#                                 'role': string},...],
#               'Latest Ranking': [{'Amount': int,
#                                'Rank': int,
#                                 'Record': string},...]}
#   'technique': {'All Technical Credits': [{' Domestic Box Office': int,
#                                          'International Box Office': int,
#                                          'Release Date': datetime,
#                                          'Role': string,
#                                          'Title': string,
#                                          'Worldwide Box Office': int},...],
#               'Director Credits': [{'Domestic Box Office': int,
#                        'Domestic Share': float,
#                        'Max Theater Count': int,
#                        'Opening Weekend Box Office': int,
#                        'Release Date': datetime,
#                        'Title': string,
#                        'Worldwide Box Office': int},...]
#               'Latest Ranking': [{'Amount': int,
#                       'Rank': int,
#                       'Record': string},...]}
#}
#
def get_info_by_person_name(name):
    with open('person_name_url.pkl', 'rb') as f:
        person_name_url = pickle.load(f)
    
    if name not in person_name_url.keys():
        return None
    
    url=person_name_url[name]
    #print(url)
    summary_url=url+"#tab=summary"
    news_url=url+"#tab=news"
    acting_url=url+"#tab=acting"
    technical_url=url+"#tab=technical"
    
    summary_response = requests.get(summary_url)
    news_response = requests.get(news_url)
    acting_response = requests.get(acting_url)
    technical_response = requests.get(technical_url)
    
    summary_dic, news_dic, acting_dic,technical_dic = dict(), dict(), dict(), dict()
    
    if summary_response.status_code != 404:
        summary_dic = parse_person_page(summary_response.content, "summary")
        #pprint(summary_dic) 
    
    if news_response.status_code != 404:
        news_dic = parse_person_page(news_response.content, "news")
        #pprint(summary_dic) 
        
    if acting_response.status_code != 404:
        acting_dic = parse_person_page(acting_response.content, "acting")
        #pprint(acting_dic) 
        
    if technical_response.status_code != 404:
        technical_dic = parse_person_page(technical_response.content, "technical")
        #pprint(technical_dic) 
        
    total_dic=dict()
    total_dic['summary']=summary_dic
    total_dic['news']=news_dic
    total_dic['acting']=acting_dic
    total_dic['technique']=technical_dic
    
    return total_dic

In [51]:
#Example 1
pprint(get_info_by_person_name('Forest Whitaker'))

{'acting': {'All Acting Credits': [{' Domestic Box Office': 0,
                                    'International Box Office': 0,
                                    'Release Date': datetime.datetime(2018, 3, 9, 0, 0),
                                    'Role': 'Desmond Tutu ',
                                    'Title': 'The Forgiven',
                                    'Worldwide Box Office': 0},
                                   {' Domestic Box Office': 681833830,
                                    'International Box Office': 642073119,
                                    'Release Date': datetime.datetime(2018, 2, 16, 0, 0),
                                    'Role': 'Zuri ',
                                    'Title': 'Black Panther',
                                    'Worldwide Box Office': 1323906949},
                                   {' Domestic Box Office': 0,
                                    'International Box Office': 0,
                                    'Rele

                               {'Amount': 99948046,
                                'Rank': 682,
                                'Record': 'Top Domestic Leading Stars for '
                                          'Original Screenplay Movies (Rank '
                                          '601-700)'},
                               {'Amount': 132094830,
                                'Rank': 230,
                                'Record': 'Top Domestic Leading Stars for '
                                          'Based on Fiction Book/Short Story '
                                          'Movies (Rank 201-300)'},
                               {'Amount': 681833830,
                                'Rank': 33,
                                'Record': 'Top Domestic Leading Stars for '
                                          'Based on Comic/Graphic Novel '
                                          'Movies'},
                               {'Amount': 146858239,
                    

                                 {' Max Theater Count': 3594,
                                  'Domestic Box Office': 89256424,
                                  'Domestic Share': 0.272,
                                  'Opening Weekend Box Office': 39201657,
                                  'Release Date': datetime.datetime(2015, 1, 9, 0, 0),
                                  'Title': 'Taken 3',
                                  'Worldwide Box Office': 327656424},
                                 {' Max Theater Count': 2101,
                                  'Domestic Box Office': 11330849,
                                  'Domestic Share': 0.7340000000000001,
                                  'Opening Weekend Box Office': 5220288,
                                  'Release Date': datetime.datetime(2013, 12, 4, 0, 0),
                                  'Title': 'Out of the Furnace',
                                  'Worldwide Box Office': 15434375},
                               

                   'month, so there is time to build buzz. If the film is as '
                   'good as it looks, it should have long legs throughout '
                   'December. Even at the low end, it will surpass Robert '
                   'Zemeckis’ previous film during its opening weekend and '
                   'should earn more domestically than that film earned '
                   'worldwide.       Bad Santa 2     Trailer: Click to Play '
                   'Official Site: BadSanta2.com Distributor: Broad Green '
                   'Pictures Release Date: November 23rd, 2016 MPAA Rating: R '
                   'for crude sexual content and language throughout, and some '
                   'graphic nudity. Source: Original Screenplay Genre: Comedy '
                   'Keywords:  Delayed Sequel, Christmas in November, '
                   'Christmas, Santa Claus Impersonators, Heist, Dwarfism, '
                   'Addiction Directed By: Mark Waters Written By: Johnny 

                   'Breaking Dawn, Part 2 opened. That film made $141 million '
                   'during its opening weekend, which is likely more than the '
                   'entire box office will make this weekend.     The Best Man '
                   'Holiday     Trailer: Click to Play Official Site: '
                   'Facebook.com/TheBestManMovie Distributor: Universal '
                   'Release Date: November 15th, 2013  MPAA Rating: R for '
                   'language, sexual content and brief nudity.  Source: '
                   'Original Screenplay / Sequel Major Genre: Romantic Comedy '
                   'Keywords: Romance, African-American, Ensemble, Delayed '
                   'Sequel, and more Directed By: Malcolm D. Lee Written By: '
                   'Malcolm D. Lee Starring: Morris Chestnut, Taye Diggs, '
                   'Regina Hall, Terrence Howard, Sanaa Lathan, Nia Long, '
                   'Harold Perrineau, Monica Calhoun, Melissa De Sousa, and

                   'than $100 million at the box office. The two films clearly '
                   "have some crossover appeal. That said, while I'm more "
                   "bullish than most analysts, I'm not willing to predict a "
                   '$100 million box office, or even a $75 million box office. '
                   'Hopefully it will do well enough to earn a profit sometime '
                   'on the home market.       A Haunted House     Trailer: '
                   'Click to Play Official Site: AHauntedHouseMovie.com '
                   'Distributor: Open Road Release Date: January 11th, 2013  '
                   'MPAA Rating: R for crude and sexual content, language and '
                   'some drug use. Source: Original Screenplay Major Genre: '
                   'Comedy Keywords: Parody, Found Footage, Hauntings, '
                   'Possession, Paranormal Investigation, Religious, Psychics, '
                   'Screenplay Written By Star, and more Dir